In [1]:
import os

import numpy as np
import pandas as pd
import PIL.Image
from IPython.display import Image
import cv2

In [2]:
os.chdir("../")
os.getcwd()

'/home/adam/Workspace/personal/unihack/fast-form'

## load dataset

In [3]:
# train_db = pd.read_csv("training_data/emnist-balanced-train.csv")
# test_db  = pd.read_csv("training_data/emnist-balanced-test.csv")

train_db = pd.read_csv("training_data/emnist-balanced-enriched-train.csv")
test_db  = pd.read_csv("training_data/emnist-balanced-enriched-test.csv")

In [4]:
with open("model_data/emnist-balanced-mapping.txt") as f:
    s = f.read()[:-1] # remove last char \n
a = [l.split(" ") for l in s.split("\n")]
res_mapper = {int(l[0]): chr(int(l[1])) for l in a}
print(res_mapper)

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I', 19: 'J', 20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'O', 25: 'P', 26: 'Q', 27: 'R', 28: 'S', 29: 'T', 30: 'U', 31: 'V', 32: 'W', 33: 'X', 34: 'Y', 35: 'Z', 36: 'a', 37: 'b', 38: 'd', 39: 'e', 40: 'f', 41: 'g', 42: 'h', 43: 'n', 44: 'q', 45: 'r', 46: 't'}


# filter only capital letters

In [5]:
def filter_letters(df):
    return df[df.iloc[:,0].apply(lambda x: 65 <= ord(res_mapper.get(x,"0")) <= 90)]

train_db = filter_letters(train_db)
test_db = filter_letters(test_db)

First, you might want to enrich the dataset - in separate notebook.

# train model

In [6]:
from keras.utils import np_utils
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.models import model_from_json

num_classes = 47
y_train = train_db.iloc[:,0]
y_train = np_utils.to_categorical(y_train, num_classes)
print ("y_train:", y_train.shape)

x_train = train_db.iloc[:,1:]
x_train = x_train.astype('float32')
x_train /= 255
print ("x_train:",x_train.shape)

inp = Input(shape=(784,))
hidden_1 = Dense(1024, activation='relu')(inp)
dropout_1 = Dropout(0.2)(hidden_1)
out = Dense(num_classes, activation='softmax')(hidden_1)
model = Model(input=inp, output=out)

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(x_train, y_train, # Train the model using the training set...
          batch_size=512, nb_epoch=10,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

y_test = test_db.iloc[:,0]
y_test = np_utils.to_categorical(y_test, num_classes)
print ("y_test:", y_test.shape)

x_test = test_db.iloc[:,1:]
x_test = x_test.astype('float32')
x_test /= 255
print ("x_test:",x_train.shape)

print(model.evaluate(x_test, y_test, verbose=1)) # Evaluate the trained model on the test set!

Using TensorFlow backend.


y_train: (124800, 47)
x_train: (124800, 784)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 112320 samples, validate on 12480 samples
Epoch 1/10
112320/112320 [==============================] - 10s 92us/step - loss: 1.1264 - accuracy: 0.6945 - val_loss: 0.9931 - val_accuracy: 0.7204
Epoch 2/10
112320/112320 [==============================] - 10s 91us/step - loss: 0.5206 - accuracy: 0.8545 - val_loss: 0.6216 - val_accuracy: 0.8276
Epoch 3/10
112320/112320 [==============================] - 10s 92us/step - loss: 0.3653 - accuracy: 0.8947 - val_loss: 0.5018 - val_accuracy: 0.8566
Epoch 4/10
112320/112320 [==============================] - 10s 90us/step - loss: 0.2911 - accuracy: 0.9152 - val_loss: 0.4393 - val_accuracy: 0.8690
Epoch 5/10
112320/112320 [==============================] - 11s 94us/step - loss: 0.2454 - accuracy: 0.9261 - val_loss: 0.3962 - val_accuracy: 0.8800
Epoch 6/10
112320/112320 [==============================] - 10s 92us/step - loss: 0.2124 - accuracy: 0.9351 - val_loss: 0.3817 - val_accuracy: 0.8813
Epoch 7/10
112320/112320 [=======================

In [7]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 91.05%


## model save/load

In [8]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
 
# serialize model to JSON
model_json = model.to_json()
with open("model_data/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_data/model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('model_data/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_data/model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy


scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], scores[1]*100))

accuracy: 91.05%
Saved model to disk
Loaded model from disk
accuracy: 91.05%


# predictions

# Display data from dataset

In [9]:
def show_letter_from_db(img):
    img = img.astype('uint8')
    img = img.reshape((28,28))
    img = img.transpose()
    img = cv2.bitwise_not(img)
    
    display(PIL.Image.fromarray(img))
    return img

In [10]:
def revert_showing(img):
    img = cv2.bitwise_not(img)
    img = img.transpose()
    img = img.ravel()
    img = img.astype('int64') 
    return img

In [11]:
def display_char(i):
    label = test_db.iloc[i].values[0]
    char = res_mapper.get(label)
    print("=============")
    print(f"label: {char}")
    
    img = test_db.iloc[i].values[1:]
    print("showing original")
    img_new=show_letter_from_db(img)
    
    img = revert_showing(img_new)
    
    pred = model.predict(np.array([img]))
    pred_val = res_mapper.get(pred[0].argmax())
    print(f"Prediction: {pred_val}")
    print("showing the one prediction run on")
    show_letter_from_db(img)

In [12]:
for i in range (10):
    display_char(i)

label: Q
showing original


Prediction: Q
showing the one prediction run on


label: X
showing original


Prediction: X
showing the one prediction run on


label: E
showing original


...

Prediction: G
showing the one prediction run on


label: R
showing original


Prediction: R
showing the one prediction run on


label: S
showing original


Prediction: S
showing the one prediction run on


# preprocessing


In [35]:
import pickle
# with open("test.txt", "wb") as fp:   #Pickling
#     pickle.dump(l, fp)

with open("./test/example_forms/julinka_dotaznik/example_letters.pickle", "rb") as fp:   # Unpickling
        b = pickle.load(fp)

len(b)


12

In [36]:
img = b[1]
display(PIL.Image.fromarray(img))

In [37]:
# preproces to training format
def preproces(img):
#     label = test_db.iloc[i].values[0]
#     char = res_mapper.get(label)ma
#     print(f"label: {char}")
    
        
    img = img.astype('float32')
    img /= 255
    
    
    pred = model.predict(np.array([img]))
    pred_val = res_mapper.get(pred[0].argmax())
    
    conf_dict = {res_mapper.get(i):round(v,3) for i,v in enumerate(pred[0])}
    conf_dict = {k: v for k, v in sorted(conf_dict.items(), key=lambda item: item[1], reverse=True) if v>0} # sort
    print(pred_val)
    print(conf_dict)
    

    
    
    img = img.astype('uint8') 
    img = img.reshape((28,28))
    img = img.transpose()
#     display(PIL.Image.fromarray(img))

In [38]:
for img in b[0:]:
    print ("=================================")
    display(PIL.Image.fromarray(cv2.bitwise_not(img)))
    preprocessed = revert_showing(img)
    preproces(preprocessed)

R
{'R': 0.992, 'B': 0.005, 'Z': 0.003}


Y
{'Y': 0.999, 'V': 0.001}


C
{'C': 0.992, 'E': 0.007}


H
{'H': 0.999, 'B': 0.001}


L
{'L': 0.941, 'K': 0.048, 'U': 0.011}


...

O
{'O': 0.986, 'G': 0.004, 'Q': 0.004, 'D': 0.003, 'C': 0.001, 'I': 0.001}


U
{'U': 1.0}


C
{'C': 0.981, 'Z': 0.01, 'I': 0.009, 'E': 0.001}


H
{'H': 1.0}


Y
{'Y': 0.921, 'T': 0.058, 'M': 0.021}


In [183]:
# img = img.astype('uint8') 

In [184]:
pred = model.predict(np.array([img]))
print(pred)
pred_val = res_mapper.get(pred[0].argmax())
print(f"Prediction: {pred_val}")

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Prediction: J


In [ ]:
bdef display_char(i):
    label = test_db.iloc[i].values[0]
    char = res_mapper.get(label)
    print("=============")
    print(f"label: {char}")
    
    img = test_db.iloc[i].values[1:]
    
    pred = model.predict(np.array([img]))
    pred_val = res_mapper.get(pred[0].argmax())
    print(f"Prediction: {pred_val}")
    
    
    img = img.astype('uint8') 
    img = img.reshape((28,28))
    img = img.transpose()
    display(PIL.Image.fromarray(img))